In [1]:
import feature_engineering as fe
import train_utils as tu
from persistent_cache import memo, PersistentDict as Perd
from sklearn.cross_validation import StratifiedKFold, KFold

In [3]:
X, XX = fe.train_test_sets("basic")
y = fe.get_y()

In [4]:
model = tu.linreg()
model.fit(X, y)
preds = model.predict(X)
offsets = tu.optimize_offsets(preds, y)

INFO:trainlo:optimising offsets 59381
INFO:trainlo:done optimising offsets 59381


In [6]:
offsets.shape

(8,)

In [7]:
model = tu.xgbr()
model.fit(X, y)
preds = model.predict(X)
offsets1 = tu.optimize_offsets(preds, y)

INFO:trainlo:optimising offsets 59381
INFO:trainlo:done optimising offsets 59381


In [8]:
offsets1.shape

(8,)

In [9]:
offsets, offsets1

(array([  0.12867712,  -2.09942806, -43.56824553,  -1.99933674,
         -1.08124142,  -0.16327151,   0.95684983,  26.11763939]),
 array([  1.28677119e-01,  -2.09942806e+00,  -4.35682455e+01,
         -1.22338917e+00,  -8.21077857e-01,   9.05535818e-03,
          7.16928157e-01,   2.61176394e+01]))

In [17]:
tu.np.vstack([offsets, offsets1]).mean(axis=0)

(8,)